In [1]:
# import packages
%matplotlib inline
import matplotlib.pyplot as plt

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, default_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

from dipy.direction import DeterministicMaximumDirectionGetter
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel

from dipy.tracking import utils
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion
from dipy.tracking.streamline import Streamlines
from dipy.viz import window, actor, colormap, has_fury

print('import complete.')

import complete.


In [2]:
# Enables/disables interactive visualization
interactive = False

# load data
hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
label_fname = get_fnames('stanford_labels')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
labels = load_nifti_data(label_fname)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

print('data loading complete.')

data loading complete.


In [3]:
# set mask(s) and seed(s)
seed_mask = labels == 2
white_matter = (labels == 1) | (labels == 2)
seeds = utils.seeds_from_mask(seed_mask, affine, density=1)

print('mask(s) and seed(s) set complete.')

mask(s) and seed(s) set complete.


In [4]:
# getting directions from diffusion dataset. 

response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)

# fit the reconstruction model with Constrained Spherical Deconvolusion (CSD)
csd_model = ConstrainedSphericalDeconvModel(gtab, response, sh_order=6)
csd_fit = csd_model.fit(data, mask=white_matter)

# define tracking mask with Constant Solid Angle (CSA)
csa_model = CsaOdfModel(gtab, sh_order=6)
gfa = csa_model.fit(data, mask=white_matter).gfa
stopping_criterion = ThresholdStoppingCriterion(gfa, .25)

print('direction computation complete. ')

direction computation complete. 


In [6]:
# start traking

# # Initialization of LocalTracking. The computation happens in the next step.
# csa_peaks = peaks_from_model(csa_model, data, default_sphere,
#                              relative_peak_threshold=.8,
#                              min_separation_angle=45,
#                              mask=white_matter)
# streamline_generator = LocalTracking(csa_peaks, stopping_criterion, seeds,
#                                       affine=affine, step_size=.5)

detmax_dg = DeterministicMaximumDirectionGetter.from_shcoeff(
    csd_fit.shm_coeff, 
    max_angle=30., 
    sphere=default_sphere)
streamline_generator = LocalTracking(detmax_dg, stopping_criterion, seeds,
                                     affine, step_size=.5)

# Generate streamlines object
streamlines = Streamlines(streamline_generator)

# sft = StatefulTractogram(streamlines, hardi_img, Space.RASMM)
# save_trk(sft, "tractogram_deterministic_dg.trk")

print('traking complete.')

traking complete.


In [11]:
# visualzie

if has_fury:
    # Prepare the display objects.
    color = colormap.line_colors(streamlines)

    streamlines_actor = actor.line(streamlines, color)

    # Create the 3D display.
    r = window.Scene()
    r.add(streamlines_actor)

    # Save still images for this static example. Or for interactivity use
    window.record(r, out_path='tractogram_deterministic_dg.png', size=(800, 800))
    
    if interactive:
        window.show(r)